- pip install

In [1]:
# 한 번만 실행
# !pip install yfinance
# !pip install ta

- import library

In [11]:
import os
import time
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import ta
import time 
# 1. stockAI 라이브러리 불러오기 - stockAI 라이브러리 개발 시 사용
# import stockAI as sai
from tqdm import tqdm  
import warnings
warnings.filterwarnings('ignore')

## 데이터 수집

### 종목 불러오기 - get_ticker
방법 1. 전체 엑셀파일 불러와서 데이터프레임으로 처리   


In [12]:
def get_ticker(markets:list):
    lst_tickers = []
    for market in markets:
#         df_ticker = pd.read_excel('tickers.xlsx')
        df_ticker = pd.read_csv('tickers.csv')
        tickers = df_ticker[df_ticker['Exchange'] == market]['Ticker'].tolist()
        lst_tickers.extend(tickers)
    return lst_tickers

- 코스피, 코스닥 

In [13]:
start_time = time.time()
lst_tickers = get_ticker(markets=['KSC', 'KOE']) # KSC: KOSPI, KOE: KOSDAQ
print("걸린 시간:", time.time() - start_time)
print(len(lst_tickers))
print(lst_tickers[:5])

걸린 시간: 0.1734626293182373
1720
['001065.KS', '096760.KS', '001067.KS', '089590.KS', '005930.KS']


- 전체 시장

In [14]:
lst_market = ['PNK', 'NMS', 'NYQ', 'ASX', 'NCM', 'NGM', 'ASE', 'JKT', 'GER', 
              'PAR', 'TOR', 'BRU', 'OBB', 'BUE', 'LSE', 'KLS', 'PCX', 'VAN',
              'AMS', 'EBS', 'TWO', 'OSL', 'STO', 'CPH', 'VIE', 'SAO', 'SES',
              'NSI', 'MEX', 'HKG', 'NZE', 'MCE', 'BSE', 'ISE', 'FRA', 'MCX',
              'MIL', 'MUN', 'ATH', 'BER', 'ENX', 'STU', 'LIS', 'TLV', 'DUS',
              'IST', 'HAM', 'HAN', 'TAL', 'TLO', 'SET', 'TAI', 'HEL', 'ICE',
              'IOB', 'CNQ', 'RIS', 'KSC', 'LIT', 'KOE', 'DOH', 'EUX', 'SHH',
              'CCS', 'MDD', 'SHZ', 'MAD']
    
start_time = time.time()
lst_tickers = get_ticker(markets=['KSC', 'KOE']) # KSC: KOSPI, KOE: KOSDAQ
print("걸린 시간:", time.time() - start_time)
print(len(lst_tickers))
print(lst_tickers[:5])

걸린 시간: 0.1728801727294922
1720
['001065.KS', '096760.KS', '001067.KS', '089590.KS', '005930.KS']


방법 2. 엑셀파일 전처리 - 마켓 별 csv파일을 따로 저장해서 파일 불러오기
- 마켓의 수가 많아질 때 속도가 더 빠름

In [15]:
def get_ticker(markets:list):
    lst_tickers = []
    for market in markets:
        df_ticker = pd.read_csv(f'tickers_by_markets/{market}.csv')
        tickers = df_ticker[df_ticker['Exchange'] == market]['Ticker'].tolist()
        lst_tickers.extend(tickers)
    return lst_tickers

- 코스피, 코스닥 

In [16]:
start_time = time.time()
lst_tickers = get_ticker(markets=['KSC', 'KOE']) # KSC: KOSPI, KOE: KOSDAQ
print("걸린 시간:", time.time() - start_time)
print(len(lst_tickers))
print(lst_tickers[:5])

걸린 시간: 0.008501529693603516
1720
['001065.KS', '096760.KS', '001067.KS', '089590.KS', '005930.KS']


- 전체 시장

In [7]:
lst_market = ['PNK', 'NMS', 'NYQ', 'ASX', 'NCM', 'NGM', 'ASE', 'JKT', 'GER', 
              'PAR', 'TOR', 'BRU', 'OBB', 'BUE', 'LSE', 'KLS', 'PCX', 'VAN',
              'AMS', 'EBS', 'TWO', 'OSL', 'STO', 'CPH', 'VIE', 'SAO', 'SES',
              'NSI', 'MEX', 'HKG', 'NZE', 'MCE', 'BSE', 'ISE', 'FRA', 'MCX',
              'MIL', 'MUN', 'ATH', 'BER', 'ENX', 'STU', 'LIS', 'TLV', 'DUS',
              'IST', 'HAM', 'HAN', 'TAL', 'TLO', 'SET', 'TAI', 'HEL', 'ICE',
              'IOB', 'CNQ', 'RIS', 'KSC', 'LIT', 'KOE', 'DOH', 'EUX', 'SHH',
              'CCS', 'MDD', 'SHZ', 'MAD']
    
start_time = time.time()
lst_tickers = get_ticker(markets=lst_market) # KSC: KOSPI, KOE: KOSDAQ
print("걸린 시간:", time.time() - start_time)
print(len(lst_tickers))
print(lst_tickers[:5])

걸린 시간: 0.14571499824523926
95092
['OEDV', 'FNMA', 'ATLS', 'ADAC', 'ASTI']


### 데이터 불러오기 - read_data
: 데이터 한번에 받아오는 코드로 수정

In [17]:
def read_data(date:list, tickers:list, market:list):
    lst_code = ' '.join(tickers)
    start = date[0]
    end = date[1]
    
#     df = yf.download(tickers=lst_code, start=start, end=end, group_by='tickers', progress=False).reset_index()
    df = yf.download(tickers=lst_code, start=start, end=end, group_by='tickers', progress=True).reset_index()
    df = df.melt(id_vars='Date') # Date로 녹이기
    df = df.pivot_table(index=['variable_0', 'Date'], columns='variable_1', values='value').reset_index()
    df = df.rename(columns={'variable_0':'Code'})
    df.columns.names= [None]
    df = df[['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    
    return df

In [18]:
# tickers = pd.read_csv('./tickers.csv')['Ticker'].tolist()
tickers=['AMZN', 'AAPL', 'GOOG']
date = ['2017-01-02', '2020-12-31']

stock_data = read_data(date=date, market=None, tickers=tickers)
stock_data

[*********************100%***********************]  3 of 3 completed


,Code,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2017-01-03,28.950001,29.082500,28.690001,29.037500,27.413372,115127600.0
1,AAPL,2017-01-04,28.962500,29.127501,28.937500,29.004999,27.382690,84472400.0
2,AAPL,2017-01-05,28.980000,29.215000,28.952499,29.152500,27.521944,88774400.0
3,AAPL,2017-01-06,29.195000,29.540001,29.117500,29.477501,27.828764,127007600.0
4,AAPL,2017-01-09,29.487499,29.857500,29.485001,29.747499,28.083660,134247600.0
...,...,...,...,...,...,...,...,...
3013,GOOG,2020-12-23,86.405502,87.399498,86.251999,86.619003,86.619003,20676000.0
3014,GOOG,2020-12-24,86.750000,87.300003,86.455498,86.942497,86.942497,6936000.0
3015,GOOG,2020-12-28,87.581749,89.536400,87.316750,88.804497,88.804497,27860000.0
3016,GOOG,2020-12-29,89.389503,89.622002,87.804497,87.935997,87.935997,25988000.0


- 코스피, 코스닥 전체 종목 

In [19]:
date = ['2017-01-02', '2020-12-31']

stock_data = read_data(date=date, market=None, tickers=lst_tickers)
stock_data

[*********************100%***********************]  1720 of 1720 completed

1080 Failed downloads:
- 042110.KQ: No data found for this date range, symbol may be delisted
- 72503A3B.KS: No data found, symbol may be delisted
- 530010.KS: No data found, symbol may be delisted
- 530007.KS: No data found, symbol may be delisted
- 060480.KQ: No data found for this date range, symbol may be delisted
- 105740.KQ: No data found for this date range, symbol may be delisted
- 090970.KS: No data found, symbol may be delisted
- 039290.KQ: No data found for this date range, symbol may be delisted
- 012860.KQ: No data found for this date range, symbol may be delisted
- 016670.KQ: No data found for this date range, symbol may be delisted
- 047920.KQ: No data found for this date range, symbol may be delisted
- 120240.KQ: No data found for this date range, symbol may be delisted
- 196490.KQ: No data found for this date range, symbol may be delisted
- 036190.KQ: No data found for this date range, symbol m

Exception in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread started byException ignored in sys.unraisablehookException in thread Exception in threading.excepthook:Exception ignored in thread s

,Code,Date,Open,High,Low,Close,Adj Close,Volume
0,000030.KS,2017-01-03,12650.0,12750.0,12550.0,12700.0,12700.0,1315387.0
1,000030.KS,2017-01-04,12700.0,12750.0,12600.0,12700.0,12700.0,1376564.0
2,000030.KS,2017-01-05,12650.0,12650.0,12250.0,12300.0,12300.0,2752633.0
3,000030.KS,2017-01-06,12300.0,12400.0,12150.0,12400.0,12400.0,1560460.0
4,000030.KS,2017-01-09,12350.0,12500.0,12250.0,12450.0,12450.0,1411935.0
...,...,...,...,...,...,...,...,...
622801,900140.KS,2020-12-23,3550.0,3610.0,3490.0,3555.0,3555.0,790508.0
622802,900140.KS,2020-12-24,3600.0,3785.0,3485.0,3610.0,3610.0,2197863.0
622803,900140.KS,2020-12-28,3575.0,3680.0,3530.0,3560.0,3560.0,879661.0
622804,900140.KS,2020-12-29,3680.0,3790.0,3645.0,3790.0,3790.0,1095775.0


In [ ]:
# 2. 데이터 수집
# df = sai.read_data(date=['2017-01-02', '2021-06-31'], market=['KOSDAQ', 'KOSPI'], ticker=['020200']) # 확인 필요

In [ ]:
# try1
'''
def read_data(date:list, tickers:list):
    """
    date: 찾고싶은 정보의 날짜 범위 
    -> ex) ["2017-01-02", "2020-12-31"] 은 2017-01-02 부터 2020-12-30 까지의 데이터
    market: (미정)
    tickers: ticker로 이루어진 list
    """
    start_date = date[0]
    end_date = date[1]
    
    dfs = pd.DataFrame()
    for ticker in tickers:
        df = yf.download(ticker, start=start_date, end=end_date)
        df = df.reset_index()
        df['Change'] = (df['Close'] - df['Close'].shift(1))/df['Close'].shift(1)*100
        df['Change'][0] = 0 # 첫 날은 0값으로 채움
        df['Ticker'] = ticker
        dfs = dfs.append(df)
    
    return dfs
'''

In [ ]:
# try2
'''
def read_data(date:list, tickers:list):
    """
    date: 찾고싶은 정보의 날짜 범위 
    -> ex) ["2017-01-02", "2020-12-31"] 은 2017-01-02 부터 2020-12-30 까지의 데이터
    market: (미정)
    tickers: ticker로 이루어진 list
    """
    start_date = date[0]
    end_date = date[1]
    
    df = yf.download(tickers, start=start_date, end=end_date)
    # melt, pivot_table
    
    dfs = pd.DataFrame()
    for ticker in df.groupby('Ticker'):
        df = df.reset_index()
        df['Change'] = (df['Close'] - df['Close'].shift(1))/df['Close'].shift(1)*100
        df['Change'][0] = 0 # 첫 날은 0값으로 채움
        df['Ticker'] = ticker
        dfs = dfs.append(df)
    
    return dfs
'''

- 데이터 저장 

In [13]:
stock_data.to_csv('raw_data_korea.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'raw_data_korea.csv'

## 데이터 전처리 

### 보조지표 추가 - add_index
- 참고 url1 https://junyoru.tistory.com/136?category=471312
- 참고 url2 https://analyzingalpha.com/yfinance-python

- ta 사용 가능 여부
    - yfinance module만 사용해야 하는지
    
- NaN값 처리 방법
    - 삭제?
    
- Adj Close 가 실제로 tradin_value 와 일치하는지 확인

- MA_ 사용자가 원하는 MA 컬럼 생성 => 기능 확장

In [20]:
def add_index(data:pd.DataFrame(), index_list:list):
    """
    data: 주가 데이터셋 (여러 종목도 가능)
    index_list: 추가하고 싶은 보조지표명이 담긴 list
    """
    import ta
    from tqdm import tqdm 
    result = pd.DataFrame()
    
    # 여러 종목의 주식 데이터 프레임에도 보조지표 추가 기능 지원할 수 있도록
    tickers = list(data['Code'].unique())
    
    for code, stock_df in tqdm(data.groupby('Code')):
#     for ticker in tickers:
#         stock_df = data[data['Code']==ticker]
#        stock_df['Code'] = code
        
        H, L, C, V = stock_df['High'], stock_df['Low'], stock_df['Close'], stock_df['Volume']
        
        stock_df['Change'] = (stock_df['Close'] - stock_df['Close'].shift(1))/stock_df['Close'].shift(1)
        
        
        add_form = {'MA5':stock_df['Close'].rolling(window = 5).mean(),
                    'MA20':stock_df['Close'].rolling(window = 20).mean(),
                    'MA60':stock_df['Close'].rolling(window = 60).mean(),
                    'MA120':stock_df['Close'].rolling(window = 120).mean(),
                    
                    'trading_value':stock_df['Close']*stock_df['Volume'],
                    'next_change':stock_df['Change'].shift(-1),
                    'prev_close':stock_df['Close'].shift(1),
                    
                    'MFI':ta.volume.money_flow_index(high=H, low=L, close=C, volume=V, fillna=True),
                    'ADI':ta.volume.acc_dist_index(high=H, low=L, close=C, volume=V, fillna=True),
                    'OBV':ta.volume.on_balance_volume(close=C, volume=V, fillna=True),
                    'CMF':ta.volume.chaikin_money_flow(high=H, low=L, close=C, volume=V, fillna=True),
                    'FI':ta.volume.force_index(close=C, volume=V, fillna=True),
                    'EOM_EMV':ta.volume.ease_of_movement(high=H, low=L, volume=V, fillna=True),
                    'VPT':ta.volume.volume_price_trend(close=C, volume=V, fillna=True),
                    'NVI':ta.volume.negative_volume_index(close=C, volume=V, fillna=True),
                    'VMAP':ta.volume.volume_weighted_average_price(high=H, low=L, close=C, volume=V, fillna=True),
                    
                    'ATR':ta.volatility.average_true_range(high=H, low=L, close=C, fillna=True),
                    'BHB':ta.volatility.bollinger_hband(close=C, fillna=True),
                    'BLB':ta.volatility.bollinger_lband(close=C, fillna=True),
                    'KCH':ta.volatility.keltner_channel_hband(high=H, low=L, close=C, fillna=True),
                    'KCL':ta.volatility.keltner_channel_lband(high=H, low=L, close=C, fillna=True),
                    'KCM':ta.volatility.keltner_channel_mband(high=H, low=L, close=C, fillna=True),
                    'DCH':ta.volatility.donchian_channel_hband(high=H, low=L, close=C, fillna=True),
                    'DCL':ta.volatility.donchian_channel_lband(high=H, low=L, close=C, fillna=True),
                    'DCM':ta.volatility.donchian_channel_mband(high=H, low=L, close=C, fillna=True),
                    'UI':ta.volatility.ulcer_index(close=C, fillna=True),
                    
                    'SMA':ta.trend.sma_indicator(close=C, fillna=True),
                    'EMA':ta.trend.ema_indicator(close=C, fillna=True),
                    'WMA':ta.trend.wma_indicator(close=C, fillna=True),
                    'MACD':ta.trend.macd(close=C, fillna=True),
                    'ADX':ta.trend.adx(high=H, low=L, close=C, fillna=True),
                    'VIneg':ta.trend.vortex_indicator_neg(high=H, low=L, close=C, fillna=True),
                    'VIpos':ta.trend.vortex_indicator_pos(high=H, low=L, close=C, fillna=True),
                    'TRIX':ta.trend.trix(close=C, fillna=True),
                    'MI':ta.trend.mass_index(high=H, low=L, fillna=True),
                    'CCI':ta.trend.cci(high=H, low=L, close=C, fillna=True),
                    'DPO':ta.trend.dpo(close=C, fillna=True),
                    'KST':ta.trend.kst(close=C, fillna=True),
                    'Ichimoku':ta.trend.ichimoku_a(high=H, low=L, fillna=True),
                    'ParabolicSAR':ta.trend.psar_down(high=H, low=L, close=C, fillna=True),
                    'STC':ta.trend.stc(close=C, fillna=True),
                    
                    'RSI':ta.momentum.rsi(close=C, fillna=True),
                    'SRSI':ta.momentum.stochrsi(close=C, fillna=True),
                    'TSI':ta.momentum.tsi(close=C, fillna=True),
                    'UO':ta.momentum.ultimate_oscillator(high=H, low=L, close=C, fillna=True),
                    'SR':ta.momentum.stoch(close=C, high=H, low=L, fillna=True),
                    'WR':ta.momentum.williams_r(high=H, low=L, close=C, fillna=True),
                    'AO':ta.momentum.awesome_oscillator(high=H, low=L, fillna=True),
                    'KAMA':ta.momentum.kama(close=C, fillna=True),
                    'ROC':ta.momentum.roc(close=C, fillna=True),
                    'PPO':ta.momentum.ppo(close=C, fillna=True),
                    'PVO':ta.momentum.pvo(volume=V, fillna=True)
                   }
        
        for index in index_list:
            stock_df[index] = add_form[index]
            
        result = result.append(stock_df)
        result = result.dropna() # 결측치가 있는 행 제거
    
    return result

In [25]:
stock_data = pd.read_csv('raw_data_korea.csv')

In [21]:
index_list = ['MA5', 'FI', 'BHB', 'BLB', 'CCI', 'next_change']
check_df = add_index(data=stock_data, index_list=index_list)
check_df

100%|█████████████████████████████████████████| 640/640 [03:53<00:00,  2.74it/s]


,Code,Date,Open,High,Low,Close,Adj Close,Volume,Change,MA5,FI,BHB,BLB,CCI,next_change
4,000030.KS,2017-01-09,12350.0,12500.0,12250.0,12450.0,12450.0,1411935.0,0.403226,12510.0,-8.636951e+07,12834.961536,12185.038464,-42.813456,1.204819
5,000030.KS,2017-01-10,12400.0,12650.0,12350.0,12600.0,12600.0,1536712.0,1.204819,12490.0,-4.110147e+07,12829.138127,12220.861873,17.391304,0.793651
6,000030.KS,2017-01-11,12550.0,12800.0,12500.0,12700.0,12700.0,1242749.0,0.793651,12490.0,-1.747627e+07,12857.059789,12242.940211,73.298429,0.787402
7,000030.KS,2017-01-12,12700.0,12850.0,12650.0,12800.0,12800.0,1351175.0,0.787402,12590.0,4.322839e+06,12912.676840,12249.823160,100.000000,-1.562500
8,000030.KS,2017-01-13,12750.0,12800.0,12500.0,12600.0,12600.0,1195682.0,-1.562500,12630.0,-3.045705e+07,12896.027717,12270.638949,35.076923,3.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622800,900140.KS,2020-12-22,3700.0,3700.0,3520.0,3540.0,3540.0,1200716.0,-4.194858,3694.0,-8.526182e+06,4032.812071,3440.687929,-83.395552,0.423729
622801,900140.KS,2020-12-23,3550.0,3610.0,3490.0,3555.0,3555.0,790508.0,0.423729,3663.0,-5.614211e+06,4022.304202,3460.695798,-107.161113,1.547117
622802,900140.KS,2020-12-24,3600.0,3785.0,3485.0,3610.0,3610.0,2197863.0,1.547117,3638.0,1.245674e+07,4008.059242,3488.940758,-74.569553,-1.385042
622803,900140.KS,2020-12-28,3575.0,3680.0,3530.0,3560.0,3560.0,879661.0,-1.385042,3592.0,4.393915e+06,4011.736712,3480.263288,-98.535019,6.460674


In [ ]:
# 저장

In [23]:
check_df.to_csv('processed_data_0912.csv')

In [ ]:
# 3. 데이터 전처리
# 3.1 보조지표 추가
# df2 = sai.add_index(data=df, index_list=['MA5'])

### 스케일링 **[new]**

스케일러 class 구현 코드 https://velog.io/@skyepodium/python-scaler-%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0

    cha0716/scaler_class.ipynb

In [24]:
def scaling(data, column_list, scaler, window_size=None):
    '''
    data: df / 스케일링 할 dataframe 
    column_list: list / 스케일링 할 컬럼 
    scaler: str / minmax, standard, robust, div-close, ...
    window_size: int or None / windowsize 또는 전체 
    '''
    from sklearn.preprocessing import StandardScaler 
    from sklearn.preprocessing import MinMaxScaler 
    from sklearn.preprocessing import RobustScaler 
    
    column_list_no_scaling = set(data.columns) - set(column_list)    
    
    df_result = pd.DataFrame([])
    for ticker, stock_df in data.groupby('Ticker'): 
        if scaler == 'standard': 
            scaler = StandardScaler()
            df_scaling = pd.DataFrame(scaler.fit_transform(stock_df[column_list]))
            df_result.append(pd.concat([df_scaling, stock_df[column_list_no_scaling]], axis=1))
            
        elif scaler == 'minmax': 
            sclaer = MinMaxScaler()
            df_scaling = pd.DataFrame(scaler.fit_transform(stock_df[column_list]))
            df_result.append(pd.concat([df_scaling, stock_df[column_list_no_scaling]], axis=1))
            
        elif scaler = 'robust':
            sclaer = RobustScaler()
            df_scaling = pd.DataFrame(scaler.fit_transform(stock_df[column_list]))
            df_result.append(pd.concat([df_scaling, stock_df[column_list_no_scaling]], axis=1))            
            
        elif scaler == 'div_close': 
            # 전 날 종가 컬럼 생성 
            stock_df['prev_close'] = stock_df['Close'].shift(1)
            stock_df.dropna(inplace=True)
            df_scaling = stock_df[column_list].apply(lambda x: x/stock_df['prev_close'])
            df_result.append(pd.concat([df_scaling, stock_df[column_list_no_scaling]], axis=1))

            
    return df_result 

SyntaxError: invalid syntax (656369708.py, line 26)

In [25]:
def scaling(data:pd.Dataframe(), column_list:list, scaler:str):
    pass

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [ ]:
# 3.2 스케일링 - 실제 column 명 or column 위치 or 둘 다(?)
# scaled_df = sai.scaling(data=df2, column_list=['MA5', 'MA20'], scaler='MinMaxScaler'

## 모델 학습 및 평가

### trader 정의 

In [ ]:
class trader: # 트레이더 저장 
    def __init__(self):
        self.name = None
        self.buyer = None
        self.seller = None

class buyer: # 매수 클래스 
    def __init__(self, sub_buyers):
        self.sub_buyers = sub_buyers
        pass
    
    def decision(self, df):
        total_amount = 1.0
        for sub in self.sub_buyers:
            amount = sub.decision(df)
            total_amount *= amount
        return total_amount
    
    def train(self, df):
        for sub in self.sub_buyers:
            if type(sub) == machinelearning_buyer:
                sub.train(df)

class conditional_buyer:
    def __init__(self):
        self.condition = None

    def decision(self, df):
        return self.condition(df)
    
    
class machinelearning_buyer:
    def __init__(self):
        self.algorithm = None
        self.threshold = None
        
    # (추가) 독립변수, 종속변수 분리 함수 
    def split(self, data:pd.DataFame(), label:str):
         '''
         label: 종속변수 조건 
                (ex) 'reg': 종속변수는 next_change
                     'class&0.02': 종속변수는 if next_change >= 0.02 1 else 0 (0또는 1)  --> 임의의 규칙 
         '''
        
        if label == 'reg': 
            X = data[]  # Code, Date, next_change 제외한 컬럼들 
            X = data.drop(["Code", "Date", "next_change"], axis = 1)
            y = data['next_change']
        
        elif 'class' in label:
            threshold = inf(label.split('&')[1]) 
            X = data[]  # Code, Date, next_change 제외한 컬럼들
            X = data.drop(["Code", "Date", "next_change"], axis = 1)
            y = (data['next_change'] >= threshold).astype('int')
            
        else:
            print("Please write a valid condition")
        
        return X, y
            
    def train(self, X, y):
        self.algorithm.fit(X, y)
    
    def set_threshold(self, threshold):
        self.threshold = threshold
    
    
    def decision(self, df):
        return self.algorithm.predict_proba(df) >= self.threshold
        
        
    
        
        
def sampling(df):
    condition1 = (-0.3 <= df.Change) & (df.Change <= 0.3) # 상한가, 하한가 초과하는 예외 제거 
    condition2 = df.trading_value >= 1000000000 # 변동성 조건 1: 거래대금 10억 이상 
    condition3 = (-0.05 >= df.Change) | (0.05 <= df.Change) # 변동성 조건 2: 금일 주가 변화율 5%이상 
    condition = condition1 & condition2 & condition3
    return condition

In [ ]:
data = pd.read_csv('processed_data.csv')

In [ ]:
for training_date, test_date in [[['2016-01-01', '2018-12-31'], ['2019-01-01', '2019-12-31']],
                                    [['2017-01-01', '2019-12-31'], ['2020-01-01', '2020-12-31']]]:
    
    # train, test dataset 분리 
    training_data = data[(data['Date'] >= training_date[0]) & (data['Date'] <= training_date[1])]
    test_data = data[(data['Date'] >= test_date[0]) & (data['Date'] <= test_date[1])]

    
    lst_trader = [] # trader 객체 저장 
    
    b1 = conditional_buyer() # 조건 매수 객체 생성
    
    def sampling(df): # 조건 함수 생성 
        condition1 = (-0.3 <= df.Change) & (df.Change <= 0.3) # 상한가, 하한가 초과하는 예외 제거 
        condition2 = df.trading_value >= 1000000000 # 변동성 조건 1: 거래대금 10억 이상 
        condition3 = (-0.05 >= df.Change) | (0.05 <= df.Change) # 변동성 조건 2: 금일 주가 변화율 5%이상 
        condition = condition1 & condition2 & condition3
        return condition
    
    b1.condition = sampling 
    
    
    b2 = machinelearning_buyer() # 머신러닝 매수 객체 생성 
    b2.algorithm = XGBoost() 
    
    trainX, trainY = b2.split(training_data) # (추가) 
    
    b2.train(trainX, trainY) # 모델 학습 
    b2.threshold = 0.8
    ######################################################## 

    
    ######################################################## 트레이더 정의  
    t = trader()
    t.name = '수원랩XGboost'
    t.buyer = buyer([b1, b2]) # conditional buyer, machinelearning buyer 
    t.seller = seller([sell_all])
    
    
    
    lst_trader.append(t)
    
    
    test_data = test_data.sort_values(key='Date')
    dic_trader_name2budget={}
    dic_trader_name2stock={}
    
    ##################################################### 매수 매도 
    # for each day
    for data in test_data.groupby('Date').iterrows:
        # for each trader
        for trader in lst_trader:
            # sell for each code
            for code in dic_trader_name2stock[trader.name]:
                for model in trader['sell_model']:
                    amount = model.decision(data[data['Code'] == code])
                    dic_trader_name2budget[trader.name] += data[data['Code']==code].close * dic_trader_name2stock[trader.name][code] * amount
                    dic_trader_name2stock[trader.name][code] *= (1-amount)
                    if dic_trader_name2stock[trader.name][code] == 0.0:
                        del dic_trader_name2stock[code]
            # buy for each code
            lst_amount = trader.decision(data)
            for code in data['Code']:
                dic_trader_name2stock[trader.name][code] += dic_trader_name2budget[trader.name]*amount / data[data['Code']==code].close
                dic_trader_name2budget[trader.name] -= dic_trader_name2budget[trader.name]*amount
                    
    ######################################################
        
        
        for model in trader.buy_model:
            test_data2 = test_data2.iloc[model.decision(test_data2), :]
            

            lst_trading_history.append((trader.name, trading_history))
            lst_yield.append((trader.name, trader.compute_yield(trading_history)))
            lst_variance.append((trader.name, trader.compute_variance(trading_history)))
        
        
    # fold 별 trader별 plot (2-fold면 2개의 큰 plot)
    plot_trading(lst_trading_history)
    plot_yield(lst_yield)
    plot_variance(lst_variance)